# certifciate_quality_classifier_v3
## This version we will try to add more and more features but still we will keep binary outcome. and we will use scipy and related package to directly call functions related to logistic regression thanks to below contributors we are able to easily calculate and run multiple ML algorithms
---
*"Scipy was created by ***Travis Oliphant, Eric Jones, and Pearu Peterson*** in 2001 as part of the effort to create a complete scientific computing environment in Python. This environment is known as the SciPy stack, and includes NumPy, matplotlib, and pandas."*

***Also learning to do beautiful formatting as above***

I have downloaded a dataset related to Wine quality lets try to convert that data into something more relevant to our study

In [29]:
#Let me convert Wine data set into Certificate data set for our study
#Importing our regular pandas to get the csv loaded

import pandas as pd
wine_df = pd.read_csv('../ref_datasets/winequality-red.csv',sep=';')
print(wine_df.head())


   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  
0      9.4        5  
1      9.8        5  
2      9.8        5 

#### oh nice! we have wine quality data, so still its quality data. so let me not try to convert it. I wil start directly with this data set, but still we need to convert Quality column into binary decision, lets calculate mean to identify a simple boundary and convert to 1 or 0 as decision.

In [30]:
print(wine_df['quality'].mean())

5.6360225140712945


#### So since mean is around 5.5, lets keep that as our boundary/threshold and apply a logic as below to have 1 if greater than 5.5 and 0 if less than 5.5

In [31]:
#importing numpy to apply the logic and create another column to store the decision
import numpy as np
wine_df['decision'] = np.where(wine_df['quality'] > 5.5, 1, 0)
print(wine_df.head())

   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  decision  
0      9.4        5         0  
1      9.8        5   

#### Now lets import ML packages and start with ML algorithm

In [32]:
#importing ML packages
import numpy as np
from scipy.optimize import minimize #we will use this to find parameters to minimize cost function we studied in v2
from sklearn.model_selection import train_test_split #we will use this to create training and test data set
from sklearn.preprocessing import StandardScaler #this we will use to standardize the feature data

# lets prepare the data (x = features and y is target)
# lets drop the original 'quality' column as it's correlated with 'decision' /
# otherwise it will become major driver in the logic, lets study how other features impact
X = wine_df.drop(['quality', 'decision'], axis=1).values
y = wine_df['decision'].values.reshape(-1, 1) # Reshape for consistency, which means it will be column vector m x 1 matrix

# split the training data and test data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

#now we will standardize the feature data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train) #fits scalar to training data and transforms - which means it will make mean = 0 and standard deviation = 1
X_test_scaled = scaler.transform(X_test) #transforms data using mean and standard deviation found in training data
# above standardization is necessary to keep same scaling for all features so that optimization happens quickly

X_train_biased = np.hstack([np.ones((X_train_scaled.shape[0], 1)), X_train_scaled]) # this will add 1 row of 1's for all features just like theta 0

# cost function defination

def cost_function(theta, X, y):
    """
    calculates cost function with theta, X and y
    """
    m = len(y)
    h = 1 / (1 + np.exp(-X @ theta)) # Sigmoid function (h(x)) as we studied yesterday
    
    # Logistic Regression Cost
    cost = (-1/m) * np.sum(y * np.log(h) + (1 - y) * np.log(1 - h))
    return cost
# gradient function defination
def gradient_function(theta, X, y):
    m = len(y)
    # Reshape theta to be a column vector
    theta_col = theta.reshape(-1, 1) 
    
    # Calculate hypothesis (h)
    h = 1 / (1 + np.exp(-X @ theta_col)) 
    
    # Calculate the error vector
    error = h - y 
    
    # Calculate the gradient: (1/m) * X_transpose @ error
    # Note: np.dot(X.T, error) gives the matrix multiplication
    gradient = (1/m) * np.dot(X.T, error)
    
    # Minimize expects the output to be a 1D array, so we flatten it
    return gradient.flatten()
# model fitting - initialize parameters
initial_theta = np.zeros((X_train_biased.shape[1])) 

# run the optimization using scipy's minimize
result = minimize(
    fun=cost_function,
    x0=initial_theta,
    args=(X_train_biased, y_train),
    method='TNC', # Truncated Newton Conjugate Gradient I will explain more in medium blog on this.
    #I had used L-BFGS-B method earlier which didnt optimize, I will explain about this in my medium blog - l = limited memory, BFGS = Broyden–Fletcher–Goldfarb–Shanno, B= Bounds,  this speeds up convergence compared to simple gradient decent
    jac=gradient_function,
    options={'maxiter': 1000} #Sets a limit on the number of optimization steps.
)

# extract optimized parameters
optimal_theta = result.x.reshape(-1, 1) 

print("SciPy Optimization Success:", result.success)
print("Optimal Logistic Regression Parameters (Theta):", optimal_theta)

C:\Users\vinay\AppData\Local\Temp\ipykernel_18292\2526607318.py:60: OptimizeWarning: Unknown solver options: maxiter
  result = minimize(


SciPy Optimization Success: False
Optimal Logistic Regression Parameters (Theta): [[ 0.00153738]
 [ 0.00103492]
 [-0.00559284]
 [ 0.00074438]
 [ 0.00048332]
 [-0.00227964]
 [ 0.00113169]
 [-0.00477754]
 [-0.00313699]
 [ 0.00150512]
 [ 0.00476019]
 [ 0.00951559]]


#### Now we found optimal paramters for each feature, which contains 11 thetas related to actual features and 1 related to bias feature.
#### let me use these optimal paramters to find the prediction and also gather accuracy of prediction

In [33]:
#lets import sklearn metrics to find accuracy, classification report and confusion matrix, remember our first v1
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# again prepare data
# add a bias term (column of 1s) to the scaled test data
X_test_biased = np.hstack([np.ones((X_test_scaled.shape[0], 1)), X_test_scaled])

# make predictions
# update optimal_theta to ensure it's a 2D column vector (p x 1) for matrix multiplication, which mean we are 12 rows and 1 column matrix
theta_col = optimal_theta.reshape(-1, 1) 

# Calculate the linear hypothesis (z (h(x) = X * theta))
z_test = X_test_biased @ theta_col

# Calculate the predicted probabilities using the Sigmoid function or logistic regression
probabilities = 1 / (1 + np.exp(-z_test))

# Convert probabilities to binary predictions (1 if probability >= 0.5, 0 otherwise)
y_pred = (probabilities >= 0.5).astype(int)

#lets evaluate the model
print("MODEL PERFORMANCE EVALUATION")

# Calculate Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy Score: {accuracy:.4f}")

# Print a detailed Classification Report
print("Classification Report:")
# y_test must be flattened to match the shape of y_pred for scikit-learn metrics
print(classification_report(y_test.flatten(), y_pred.flatten(), target_names=['Reject (0)', 'Approve (1)']))

# Print the Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


# Interpretting model co-efficients
print("MODEL INTERPRETATION")
feature_names = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 
                 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 
                 'density', 'pH', 'sulphates', 'alcohol']

# The first coefficient is the intercept (theta_0)
intercept = optimal_theta[0][0]

# Pair the remaining coefficients with their feature names
# We use optimal_theta[1:] because index 0 is the intercept
coefficients_data = list(zip(feature_names, optimal_theta[1:].flatten()))

print(f"Model Intercept (Bias): {intercept:.4f}")
print("Feature Coefficients")
# Sort coefficients by absolute magnitude for easy interpretation of importance
for name, coef in sorted(coefficients_data, key=lambda item: abs(item[1]), reverse=True):
    # Print a positive or negative sign for clarity
    print(f"  {name:20}: {coef:+.4f}")

MODEL PERFORMANCE EVALUATION
Accuracy Score: 0.7229
Classification Report:
              precision    recall  f1-score   support

  Reject (0)       0.66      0.76      0.71       213
 Approve (1)       0.78      0.69      0.74       267

    accuracy                           0.72       480
   macro avg       0.72      0.73      0.72       480
weighted avg       0.73      0.72      0.72       480

Confusion Matrix:
[[162  51]
 [ 82 185]]
MODEL INTERPRETATION
Model Intercept (Bias): 0.0015
Feature Coefficients
  alcohol             : +0.0095
  volatile acidity    : -0.0056
  total sulfur dioxide: -0.0048
  sulphates           : +0.0048
  density             : -0.0031
  chlorides           : -0.0023
  pH                  : +0.0015
  free sulfur dioxide : +0.0011
  fixed acidity       : +0.0010
  citric acid         : +0.0007
  residual sugar      : +0.0005


### With above results:

accuracy = 0.7229 = 72.29%

78% of time it predicted correctly for 1's (approved)

76% of time it predicted correctly for 0's (rejected)

Model is better in predicting approve then reject (impleied with f1 score)

macro avg provides that both approve and rejects were balanced and didnt observe any bias

### confusion matrix:
    
[TN, FP]
[FN, TP]

True Negatives (162): Correct prediction= 162 

False Positives (51): Incorrect prediction= 51 

False Negatives (82): Incorrect prediction= 82 

True Positives (185): Correct prediction= 185 

## We were able to achieve 72 % accuracy with multiple features, we will try to upversion this to improve the accuracy by experimenting with:

-Featuring Engineering
-

-Regularization
-

-Data Balancing and Threshold Tuning
-

-Alternative Algorithms such as : Support Vector Machines (SVM), Random Forest or Gradient Boosting
-
